# Pitch

In [1]:
import penaltyblog as pb
import numpy as np
import pandas as pd

In [6]:
df = pd.DataFrame({
    'x': np.random.uniform(0, 100, 11),
    'y': np.random.uniform(0, 100, 11),
    'name': [f'Player {i+1}' for i in range(11)]
})

pitch = pb.viz.Pitch(width=800, height=600, theme="minimal", provider="wyscout")
pitch.plot_scatter(df, x='x', y='y', hover='name')
pitch.show()


In [3]:
df2 = pd.DataFrame({
    'x': np.random.uniform(0, 120, 500),
    'y': np.random.uniform(0, 80, 500)
})

# Create the pitch and plot the heatmap
pitch = pb.viz.Pitch(theme="night")
pitch.plot_heatmap(df2, x='x', y='y')
pitch.show()

In [4]:
data = pd.DataFrame({
    'x': [30, 50, 70],
    'y': [40, 60, 20],
    'x2': [45, 65, 80],
    'y2': [50, 70, 30],
    'player': ['Alice', 'Bob', 'Carol']
})

# Create a pitch with a dark theme and Opta dimensions
pitch = pb.viz.Pitch(theme="white")

# Plot the passes as arrows
pitch.plot_arrows(
    data, x='x', y='y', x_end='x2', y_end='y2'
)

# Optional: plot player positions at the start of the arrows
pitch.plot_scatter(data, x='x', y='y', hover='player')

# Show the pitch
pitch.show()


In [7]:
df = pd.DataFrame({
    'x': [10, 80, 70],
    'y': [10, 60, 20],
    'x2': [40, 65, 80],
    'y2': [50, 70, 30],
    'player': ['Alice', 'Bob', 'Carol']
})

pitch = pb.viz.Pitch(width=800, height=600, theme="retro", provider="wyscout",)
pitch.plot_comets(df, x='x', y='y', x_end='x2', y_end='y2', width=4, segments=20,hover='player')
pitch.show()